In [24]:
import pygsti
import numpy as np
from scipy.linalg import expm
from pygsti.circuits import Circuit
from matplotlib import pyplot as plt

In [25]:
# Gell-Mann matrices
gellmann_matrices = [
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]),
    np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]),
    np.array([[0, -1j, 0], [1j, 0, 0], [0, 0, 0]]),
    np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]]),
    np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]]),
    np.array([[0, 0, -1j], [0, 0, 0], [1j, 0, 0]]),
    np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]),
    np.array([[0, 0, 0], [0, 0, -1j], [0, 1j, 0]]),
    np.array([[1, 0, 0], [0, 1, 0], [0, 0, -2]])
]

gellmann_8_12 = np.array([[-2, 0, 0], [0, 1, 0], [0, 0, 1]])

In [26]:
# unitary models 
# we ignore axis error 
def modelX01(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[1] + gamma*gellmann_matrices[8]))

def modelZ01():
    return expm(-(1j*np.pi/4)*gellmann_matrices[3])

Z12_gen = np.array([[0, 0, 0], [0, 1, 0], [0, 0, -1]])

def modelZ12():
    return expm(-(1j*np.pi/4)*Z12_gen)

def modelX12(theta, gamma):
    return expm(-(1j/2)*((np.pi/2 + theta)*gellmann_matrices[4]  + gamma*gellmann_8_12))

def modelCZ(phis):
    return np.diag([1]+[np.exp(-1j*phi) for phi in phis])



In [27]:
def parse_error_vector(x):
    info = {
        'single_qutrit': {
            'Q1': {
                'X01' : x[0],
                'phase01': x[1],
                'X12' : x[2], 
                'phase12': x[3],
            },
            'Q2': {
                'X01' : x[4],
                'phase01': x[5],
                'X12' : x[6],
                'phase12': x[7],
            }
        },
        'two_qutrit': {
            'phi1': x[8],
            'phi2': x[9],
            'phi3': x[10],
            'phi4': x[11],
            'phi5': x[12],
            'phi6': x[13],
            'phi7': x[14],
            'phi8': x[15]
        }  
    }
    return info

def random_error_vector(single_qutrit_rates, two_qutrit_rates):
    q1_vec = np.random.multivariate_normal(np.zeros(4), np.eye(4)*single_qutrit_rates)
    q2_vec = np.random.multivariate_normal(np.zeros(4), np.eye(4)*single_qutrit_rates)
    two_qubit_vec = np.random.multivariate_normal(np.zeros(8), np.eye(8)*two_qutrit_rates)
    return np.concatenate((q1_vec, q2_vec, two_qubit_vec))


In [28]:
from pygsti.tools import unitary_to_std_process_mx
from pygsti.modelmembers.operations import EmbeddedOp, FullArbitraryOp

In [29]:
from pygsti.baseobjs import ExplicitStateSpace
from pygsti.models import ExplicitOpModel
from pygsti.models.modelconstruction import create_spam_vector
from pygsti.modelmembers.povms import UnconstrainedPOVM, FullPOVMEffect
from pygsti.modelmembers.states import FullState
from pygsti.tools import change_basis

psi0 = np.array([1, 0, 0])
psi1 = np.array([0, 1, 0])
psi2 = np.array([0, 0, 1])

def label_to_spam_vec(lbl):
    if lbl == '00':
        rho = np.diag([1, 0, 0, 0, 0, 0, 0, 0, 0])
    elif lbl == '01':
        rho = np.diag([0, 1, 0, 0, 0, 0, 0, 0, 0])
    elif lbl == '02':
        rho = np.diag([0, 0, 1, 0, 0, 0, 0, 0, 0])
    elif lbl == '10':
        rho = np.diag([0, 0, 0, 1, 0, 0, 0, 0, 0])
    elif lbl == '11':
        rho = np.diag([0, 0, 0, 0, 1, 0, 0, 0, 0])
    elif lbl == '12':
        rho = np.diag([0, 0, 0, 0, 0, 1, 0, 0, 0])
    elif lbl == '20':
        rho = np.diag([0, 0, 0, 0, 0, 0, 1, 0, 0])
    elif lbl == '21':
        rho = np.diag([0, 0, 0, 0, 0, 0, 0, 1, 0])
    elif lbl == '22':
        rho = np.diag([0, 0, 0, 0, 0, 0, 0, 0, 1])
    return change_basis(rho.reshape(-1, 1).astype(float), 'std', 'gm')

def make_model(error_vector, single_qutrit_depol, two_qutrit_depol):
    joint_state_space = ExplicitStateSpace([('Q1','Q2')], [(3,3)])
    ss_q1 = ExplicitStateSpace('Q1', 3)
    ss_q2 = ExplicitStateSpace('Q2', 3)

    model = ExplicitOpModel(joint_state_space)

    # Add state prep and measurement
    
    model['rho0'] =  change_basis(np.diag([1, 0, 0, 0, 0, 0, 0, 0, 0]).reshape(-1, 1).astype(float), 'std', 'gm')
    model['Mdefault'] = UnconstrainedPOVM({ '00' : label_to_spam_vec('00'),
                                            '01' : label_to_spam_vec('01'),
                                            '02' : label_to_spam_vec('02'),
                                            '10' : label_to_spam_vec('10'),
                                            '11' : label_to_spam_vec('11'),
                                            '12' : label_to_spam_vec('12'),
                                            '20' : label_to_spam_vec('20'),
                                            '21' : label_to_spam_vec('21'),
                                            '22' : label_to_spam_vec('22')}, evotype='densitymx')

    # Parse error vector
    errors = parse_error_vector(error_vector)
    x01_Q1 = errors['single_qutrit']['Q1']['X01']
    x12_Q1 = errors['single_qutrit']['Q1']['X12']
    x01_Q2 = errors['single_qutrit']['Q2']['X01']
    x12_Q2 = errors['single_qutrit']['Q2']['X12']
    
    phase01_Q1 = errors['single_qutrit']['Q1']['phase01']
    phase12_Q1 = errors['single_qutrit']['Q1']['phase12']
    phase01_Q2 = errors['single_qutrit']['Q2']['phase01']
    phase12_Q2 = errors['single_qutrit']['Q2']['phase12']

    phi1 = errors['two_qutrit']['phi1']
    phi2 = errors['two_qutrit']['phi2']
    phi3 = errors['two_qutrit']['phi3']
    phi4 = errors['two_qutrit']['phi4']
    phi5 = errors['two_qutrit']['phi5']
    phi6 = errors['two_qutrit']['phi6']
    phi7 = errors['two_qutrit']['phi7']
    phi8 = errors['two_qutrit']['phi8']
    phis = [phi1, phi2, phi3, phi4, phi5, phi6, phi7, phi8]


    # Define single qutrit gates
    X01_Q1_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelX01(x01_Q1, phase01_Q1)), 'std', 'gm'), basis='gm', state_space=ss_q1)
    Z01_Q1_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelZ01()), 'std', 'gm'), basis='std', state_space=ss_q1)
    X12_Q1_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelX12(x12_Q1, phase12_Q1)), 'std', 'gm'), basis='gm', state_space=ss_q1)
    Z12_Q1_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelZ12()), 'std', 'gm'), basis='gm', state_space=ss_q1)

    X01_Q1_unitary.depolarize(single_qutrit_depol)
    Z01_Q1_unitary.depolarize(single_qutrit_depol)
    X12_Q1_unitary.depolarize(single_qutrit_depol)
    Z12_Q1_unitary.depolarize(single_qutrit_depol)

    X01_Q2_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelX01(x01_Q2, phase01_Q2)), 'std', 'gm'), basis='gm', state_space=ss_q2)
    Z01_Q2_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelZ01()), 'std', 'gm'), basis='std', state_space=ss_q2)
    X12_Q2_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelX12(x12_Q2, phase12_Q2)), 'std', 'gm'), basis='gm', state_space=ss_q2)
    Z12_Q2_unitary = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelZ12()), 'std', 'gm'), basis='gm', state_space=ss_q2)

    X01_Q2_unitary.depolarize(single_qutrit_depol)
    Z01_Q2_unitary.depolarize(single_qutrit_depol)
    X12_Q2_unitary.depolarize(single_qutrit_depol)
    Z12_Q2_unitary.depolarize(single_qutrit_depol)

    # Define two qutrit gates
    CZ = FullArbitraryOp(change_basis(unitary_to_std_process_mx(modelCZ(phis)), 'std', 'gm'), basis='gm', state_space=joint_state_space)
    CZ.depolarize(two_qutrit_depol)

    # Add gates to model
    model.operations[('G_X01', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), X01_Q1_unitary)
    model.operations[('G_Z01', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), Z01_Q1_unitary)
    model.operations[('G_X12', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), X12_Q1_unitary)
    model.operations[('G_Z12', 'Q1')] = EmbeddedOp(joint_state_space, ('Q1',), Z12_Q1_unitary)
    model.operations[('G_X01', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), X01_Q2_unitary)
    model.operations[('G_Z01', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), Z01_Q2_unitary)
    model.operations[('G_X12', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), X12_Q2_unitary)
    model.operations[('G_Z12', 'Q2')] = EmbeddedOp(joint_state_space, ('Q2',), Z12_Q2_unitary)

    model.operations['G_CZ'] = CZ

    return model

In [30]:
prep_dict = {
    '00' : ([]), 
}

meas_dict = {
    '00' : ([]),
}


gate_dict = {
    'G_CZ' : ([('Gcz', 'Q1', 'Q2')]),
    'G_X01' : ([('G_X01(Q1)', 'Q1')]),
    'G_Z01' : ([('G_Z01(Q1)', 'Q1')]),
    'G_X12' : ([('G_X12(Q1)', 'Q1')]),
    'G_Z12' : ([('G_Z12(Q1)', 'Q1')]),
    'G_X01' : ([('G_X01(Q2)', 'Q2')]),
    'G_Z01' : ([('G_Z01(Q2)', 'Q2')]),
    'G_X12' : ([('G_X12(Q2)', 'Q2')]),
    'G_Z12' : ([('G_Z12(Q2)', 'Q2')]),
}

def make_circuit(gate, prep_label, meas_label, depth):
    prep_circ = prep_dict[prep_label]
    meas_circ = meas_dict[meas_label]
    return Circuit(['rho0'] + prep_circ + [gate]*depth + meas_circ + ['Mdefault'], line_labels=['Q1', 'Q2'])

In [17]:
x = random_error_vector(0, 0)
model = make_model(x, 0.001, 0.01)

In [18]:
model['Mdefault']['00'].to_dense()

array([0.33333333, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.70710678, 0.40824

In [19]:
circ = make_circuit(('G_X01', 'Q2'), '00', '00', 1)

In [20]:
print(circ)

Qubit Q1 ---|rho0|-|     |-|Mdefault|---
Qubit Q2 ---|rho0|-|G_X01|-|Mdefault|---



In [21]:
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.8414517253927576),
                  (('01',), -0.15754827460724233),
                  (('02',), 0.12224887852533313),
                  (('10',), 0.18217301226998187),
                  (('11',), 0.39633550798583994),
                  (('12',), -0.08773095591996724),
                  (('20',), -0.21307127424204034),
                  (('21',), -0.08396973051577436),
                  (('22',), 0.00011111111111102023)])

In [542]:
model.operations

OrderedMemberDict([(Label(('G_X01', 'Q1')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19ded0>),
                   (Label(('G_Z01', 'Q1')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19cc10>),
                   (Label(('G_X12', 'Q1')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19c0d0>),
                   (Label(('G_Z12', 'Q1')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19ffa0>),
                   (Label(('G_X01', 'Q2')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19c850>),
                   (Label(('G_Z01', 'Q2')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19c460>),
                   (Label(('G_X12', 'Q2')),
                    <pygsti.modelmembers.operations.embeddedop.EmbeddedOp at 0x7fbffc19c580>),
                   (Label((